In [1]:
import pandas as pd
import csv
import numpy as np
import os
import glob
from scipy.stats import ranksums
from statsmodels.sandbox.stats.multicomp import multipletests

# Here we measure the significant difference between the methylated value in tumor and normal tissue by considereding regions as differentially methylated based on the Wilcoxon rank-sum test adjusted byBenjamini-Hochberg method

In [2]:
names = ["TCGA-BRCA","TCGA-GBM","TCGA-OV","TCGA-LUAD","TCGA-UCEC","TCGA-KIRC","TCGA-HNSC","TCGA-LGG","TCGA-THCA","TCGA-LUSC","TCGA-PRAD","TCGA-SKCM","TCGA-COAD","TCGA-STAD","TCGA-BLCA","TCGA-LIHC","TCGA-CESC","TCGA-KIRP","TCGA-SARC","TCGA-LAML","TCGA-PAAD","TCGA-ESCA","TCGA-PCPG","TCGA-READ","TCGA-TGCT","TCGA-THYM","TCGA-KICH","TCGA-ACC","TCGA-MESO","TCGA-UVM","TCGA-DLBC","TCGA-UCS","TCGA-CHOL"]
dirFile = 'cancer_types/'
p_values = []
for i in range(len(names)):
    print(names[i])
    search_path = dirFile + names[i] + "/myth/*.txt"
    allFile = glob.glob(search_path)
    for file_ in allFile:
        temp_norml = my_df.loc[my_df['Case ID'] == my_df['Case ID'] and my_df['Sample Type'] == 'Solid Tissue Normal']
        temp_tumor = my_df.loc[my_df['Case ID'] == my_df['Case ID'] and my_df['Sample Type'] == 'Primary Tumor']
        df_normal = pd.read_csv(dirFile+names[i]+"/myth/"+temp_norml['File Name'], header=None, sep='\t')
        df_tumor = pd.read_csv(dirFile+names[i]+"/myth/"+temp_tumor['File Name'], header=None, sep='\t')
        df_normal.columns = ['Transcript_ID','Beta_value']
        df_tumor.columns = ['Transcript_ID','Beta_value']
        for index, row in df_normal.iterrows():
            s, p = ranksums(row['Beta_value'],df_tumor['Beta_value'][index])
            p_values.append(p)
        df = pd.DataFrame(p_values, columns=['p-value'])
        p_adjusted = multipletests(df['p-value'], method='fdr_bh')
        dataframe=pd.DataFrame(p_adjusted[1], columns=['p_adjusted']) 
        df = dataframe[['p_adjusted']]
        fin = pd.concat([df_normal['Transcript_ID'], df], axis=1)
        fin.to_csv(dirFile+names[i]+"/myth/"+file_, sep='\t', index=False, header=None)